In [1]:
import pandas as pd

def clean_and_reshape_unemployment_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Defining the list of the 7 states
    state_list = ['New South Wales', 'Victoria', 'Queensland', 'South Australia', 
                  'Western Australia', 'Tasmania', 'Northern Territory']
    
    # Filter columns by checking for the specific states and "Unemployment rate" in the name
    selected_cols = ['Unnamed: 0']  # Assuming 'Unnamed: 0' is the Date column
    for state in state_list:
        for col in df.columns:
            if f'Unemployment rate ;  Persons ;  > {state} ;' in col and '.1' not in col and '.2' not in col:
                selected_cols.append(col)
    
    # Select only the relevant columns
    df_selected = df[selected_cols]
    
    # Rename the columns to more descriptive names
    df_selected.columns = ['Date', 'NSW Unemployment Rate', 'VIC Unemployment Rate', 
                           'QLD Unemployment Rate', 'SA Unemployment Rate', 
                           'WA Unemployment Rate', 'TAS Unemployment Rate', 
                           'NT Unemployment Rate']

    # Remove rows with specific keywords
    rows_to_remove = ['Unit', 'Series Type', 'Data Type', 'Series Start', 
                      'Series End', 'No. Obs', 'Series ID']
    df_cleaned = df_selected[~df_selected['Date'].isin(rows_to_remove)]
    
    # Convert 'Date' column into datetime format
    df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], format='%b-%Y', errors='coerce')

    # Drop rows where the 'Date' conversion failed (NaT)
    df_cleaned = df_cleaned.dropna(subset=['Date'])

    # Reshape the data into long format
    df_long = pd.melt(df_cleaned, id_vars=['Date'], 
                      var_name='State', 
                      value_name='Unemployment Rate')

    # Clean the 'State' column (remove extra words)
    df_long['State'] = df_long['State'].str.replace(' Unemployment Rate', '')
    
    # Remove any duplicate or redundant rows
    df_long = df_long.drop_duplicates()

    return df_long

# Example usage
file_path = 'data/unemploymentRateTimeSeries.csv'  # Replace with the actual file path
reshaped_data = clean_and_reshape_unemployment_data(file_path)

# Save the reshaped data to a new CSV file if needed
reshaped_data.to_csv('data/unemploymentRateTimeSeries.csv', index=False)

# Display the reshaped data
print(reshaped_data.head())


        Date State Unemployment Rate
0 1978-02-01   NSW               6.6
1 1978-03-01   NSW               6.5
2 1978-04-01   NSW               6.4
3 1978-05-01   NSW               6.3
4 1978-06-01   NSW               6.3


/var/folders/qd/69s8gt4d5rn66njn9cz1rb5r0000gn/T/ipykernel_43440/2258130083.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], format='%b-%Y', errors='coerce')


   Date       NSW Unemployment Rate VIC Unemployment Rate  \
7  1978-02-01  6.6                   5.7                    
8  1978-03-01  6.5                   5.7                    
9  1978-04-01  6.4                   5.6                    
10 1978-05-01  6.3                   5.6                    
11 1978-06-01  6.3                   5.5                    

   QLD Unemployment Rate SA Unemployment Rate WA Unemployment Rate  \
7   7.1                   6.5                  6.2                   
8   7.1                   6.5                  6.2                   
9   7.1                   6.6                  6.3                   
10  7.1                   6.7                  6.4                   
11  7.1                   6.9                  6.5                   

   TAS Unemployment Rate NT Unemployment Rate  
7   6.3                   6.4                  
8   6.3                   6.1                  
9   6.4                   5.9                  
10  6.4             